# Let's load a dataset and try training!

To execute this notebook, you must have followed the steps in init.ipynb in data/ in order to have the data downloaded.

In [1]:
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset,random_split
from einops import rearrange
from torch.optim import Adam
from pathlib import Path
from torchvision import transforms
import matplotlib.pyplot as plt
import cv2
import sys

sys.path.append('../')
from scripts import forward_process as fp, reverse_process as rp, train, util
from scripts import customUNet as unet

In [2]:
# Load the dataset
ab1 = np.load("../data/image-colorization/ab/ab/ab1.npy")
ab2 = np.load("../data/image-colorization/ab/ab/ab2.npy")
ab3 = np.load("../data/image-colorization/ab/ab/ab3.npy")
l = np.load("../data/image-colorization/l/gray_scale.npy")

ab = np.concatenate((ab1,ab2,ab3),axis=0)

def create_lab(n):
    lab= np.zeros((n,224,224,3))
    lab[:,:,:,0] = l[0:n:]
    lab[:,:,:,1:] = ab[0:n:]  
    return lab.astype("uint8")

## Let's load training dataset

Let's load only 2000 images here (our computer can't handle more in memory). The rest of the dataset can then be used for training. We are aware that much more data is necessary to train such a big model, and that we should have train/test/val sets with much more data in the train set. But for lack of time, we will try to train the model on 2000 images to produce results.

In [ ]:
n=2000
size=64
batch_size=25

lab = create_lab(n)

# Resize and normalize pictures between -1 and 1
resized = []
for i in lab:
    img = cv2.normalize(i, None, alpha = -1, beta = 1, norm_type = cv2.NORM_MINMAX, dtype = cv2.CV_32F)
    resized.append(cv2.resize(img, dsize=(size, size), interpolation=cv2.INTER_CUBIC))

lab = torch.tensor(np.array(resized))
# Reshape so the tensor has the right shape for training
lab = rearrange(lab,"b h w c-> b c h w")
lab.shape

In [4]:
# Extract the grayscale (l) color channel
gray = lab[:,0,:,:,None]
# Tripple the last dimension so it is the same shape as the color image
gray = gray.expand(gray.shape[0],gray.shape[1],gray.shape[2],3)
gray = rearrange(gray,"b h w c-> b c h w")
gray.shape

torch.Size([2000, 3, 64, 64])

In [5]:
# Create pairs of data (lab,gray)
data=[]
for i in range(len(lab)):
    data.append([lab[i],gray[i]])

In [1]:
# Load it in a dataloader 
dataloader = DataLoader(data, batch_size=batch_size, shuffle=True)  

NameError: name 'DataLoader' is not defined

# Start training

In [7]:
T = 300
epochs = 5
model = unet.CustomConditionalUNet()
diffTerms = util.ConstantDiffusionTerms(T, fp.linear_beta_schedule)
device = util.set_device()
model.to(device)
optimizer = Adam(model.parameters(), lr=1e-3)

7303971


In [8]:
train.train(epochs,device,optimizer,dataloader,batch_size,T, diffTerms,model)

Step 0 of epoch 0: Loss=0.9751895070075989
Step 10 of epoch 0: Loss=0.8038657903671265
Step 20 of epoch 0: Loss=0.7992662191390991


KeyboardInterrupt: 